# Q5


In [7]:
import autograd.numpy as np
from autograd import elementwise_grad
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_digits
from sklearn.preprocessing import normalize


class NeuralNetwork():
    def __init__(self, hidden_layers , activation_layer, X ,y, flag,alpha):
        self.alpha=alpha
        self.X = np.array(X)
        self.y = np.array(y)
        self.flag = flag
        self.hidden_layers = hidden_layers
        self.activation_layer = activation_layer
        self.W = []
        self.B = []
        self.W.append(np.ones((hidden_layers[0],X.shape[1])))
        for i in range(len(hidden_layers)): 
            if i==len(hidden_layers)-1:
                if flag==0:
                    self.W.append(np.ones((1,hidden_layers[-1])))
                else:
                    self.W.append(np.ones((len(np.unique(y)),hidden_layers[-1])))
            else:
                self.W.append(np.ones((hidden_layers[i+1],hidden_layers[i])))

        for i in range(len(hidden_layers)+1):
            if i==len(hidden_layers):
                self.B.append(np.zeros(len(np.unique(y))))
            else:
                self.B.append(np.zeros(hidden_layers[i]))

    def grad_func(self,A,y):
        A = np.exp(A)
        A = A/A.sum(axis = 0)
        return -1* np.log(A[y])
    
    def activation_funtion(self,Z,function):
        if function == 'relu':
            return np.maximum(0,Z)
        elif function == 'sigmoid':
            return 1/(1+np.exp(-1*Z))
        elif function =="identity":
            return Z
        else:
          print("Wrong Activation function")

    def forward(self, X):
        A = np.array(X)
        for i in range(len(self.W)):
            Z = np.dot(self.W[i],A.T)
            for j in range(Z.shape[0]):
                for k  in range(Z.shape[1]):
                    Z[j,k] = Z[j,k] + self.B[i][j]
            A = self.activation_funtion(Z,self.activation_layer[i]).T
        if self.flag:
            return np.exp(A)/np.exp(A).sum(axis=0)
        return A

    def backward(self,X,y):
        def cost(inp,c):
            if c==1:
                B=self.B
                W=inp
            else:
                B=inp
                W=self.W
            x = np.array(X)
            for i in range(len(W)):
                Z = np.matmul(W[i],x.T) + B[i]
                x = self.activation_funtion(Z,self.activation_layer[i]).T
            if self.flag:
                return self.grad_func(x,y)
            return np.square(y-x)

        gradient = elementwise_grad(cost)
        grad_wrt_w = gradient(self.W,1)
        for i in range(len(self.W)):
            self.W[i] = self.W[i] - self.alpha * grad_wrt_w[i] 
        grad_wrt_b = gradient(self.B,0)
        for i in range(len(self.B)):
            self.B[i] = self.B[i] - self.alpha * grad_wrt_b[i]



# Q6
## Digits Dataset

In [11]:
def accuracy(y,y_hat):
    y_hat_new = []
    for i in range(len(y_hat)):
        index=0
        for j in range(len(y_hat[0])):
            if y_hat[i,j]>=y_hat[i,index]:
                index = j
        y_hat_new.append(index)
    return accuracy_score(y,y_hat_new)

data = load_digits()
X = data.data
X = normalize(X)
y = data.target
avg_accuracy=0
layer=[5,6]
kf3 = KFold(n_splits=3, shuffle=False)
for train_index, test_index in kf3.split(X):
    NN = NeuralNetwork(layer, ['sigmoid','relu','sigmoid'], X,y,1,alpha=0.1)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    for i in range(5):
        for j in range(X_train.shape[0]):
            NN.backward(X_train[j], y_train[j])
    y_hat = NN.forward(X_test)
    avg_accuracy += accuracy(y_test,y_hat)
print("Average Accuracy" ,avg_accuracy/3)

Average Accuracy 0.1001669449081803


## On Bostan Dataset

In [10]:
from sklearn.datasets import load_boston
data = load_boston()
X = data.data
Y = data.target
X = normalize(X)
layer = [13, 10]
rmse = 0
kf3 = KFold(n_splits=3, shuffle=False)
for train_index, test_index in kf3.split(X):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    NN = NeuralNetwork(layer, ['relu','relu', 'relu'], X,y,0,alpha=0.1)
    for i in range(5):
        for j in range(X_train.shape[0]):
            NN.backward(X_train[j], y_train[j])
    y_hat = NN.forward(X_test)
    n=X_test.shape[0]
    for i in range(n):
        rmse += np.sqrt(np.square(y_hat[i][0]-y_test[i])) /n
  
print("Average Rmse value" ,rmse/3)

Average Rmse value 2.7226659406774427
